In [1]:
%pwd

'e:\\Machine-Learning-ChatBot\\experiments'

In [2]:
%cd ..

e:\Machine-Learning-ChatBot


c:\Users\DELL\anaconda3\envs\firstbot\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%pwd

'e:\\Machine-Learning-ChatBot'

In [11]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [14]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
extracted_data = load_pdf_file(data="data/")

In [16]:
extracted_data

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-03-27T17:52:49-04:00', 'author': 'Tiep Vu', 'title': 'Machine Learning co ban', 'subject': '', 'keywords': '', 'moddate': '2018-03-27T17:52:49-04:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'source': 'data\\book_ML_color.pdf', 'total_pages': 400, 'page': 0, 'page_label': '1'}, page_content='Vũ Hữu Tiệp \nMachine Learning \ncơ bản \nmachinelearningcoban.com'),
 Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-03-27T17:52:49-04:00', 'author': 'Tiep Vu', 'title': 'Machine Learning co ban', 'subject': '', 'keywords': '', 'moddate': '2018-03-27T17:52:49-04:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'source': 'data\\book_ML_color.pdf', 'to

In [17]:
text_chunks = text_split(extracted_data)

In [18]:
len(text_chunks)

533

In [44]:
text_chunks

[Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-03-27T17:52:49-04:00', 'author': 'Tiep Vu', 'title': 'Machine Learning co ban', 'subject': '', 'keywords': '', 'moddate': '2018-03-27T17:52:49-04:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'source': 'data\\book_ML_color.pdf', 'total_pages': 400, 'page': 0, 'page_label': '1'}, page_content='Vũ Hữu Tiệp \nMachine Learning \ncơ bản \nmachinelearningcoban.com'),
 Document(metadata={'producer': 'pdfTeX-1.40.17', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-03-27T17:52:49-04:00', 'author': 'Tiep Vu', 'title': 'Machine Learning co ban', 'subject': '', 'keywords': '', 'moddate': '2018-03-27T17:52:49-04:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'source': 'data\\book_ML_color.pdf', 'to

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [6]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [7]:
from langchain.embeddings import OpenAIEmbeddings

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

C:\Users\DELL\AppData\Local\Temp\ipykernel_9524\3280910429.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [39]:
query_embedding = embeddings.embed_query("Hợp đồng lao động là gì?")

In [40]:
len(query_embedding)

1536

In [9]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mlbot"


pc.create_index(
    name=index_name,
    dimension=1536, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

c:\Users\DELL\anaconda3\envs\firstbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [19]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [40]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [39]:
docsearch

In [41]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [23]:
retrieved_docs = retriever.invoke("Làm sao để đánh giá hiệu suất của mô hình trên tập dữ liệu mới?")

In [24]:
retrieved_docs

[Document(id='db691f5e-961b-4083-a1d0-0f07fcb80021', metadata={'author': 'Tiep Vu', 'creationdate': '2018-03-27T17:52:49-04:00', 'creator': 'LaTeX with hyperref package', 'keywords': '', 'moddate': '2018-03-27T17:52:49-04:00', 'page': 78.0, 'page_label': '67', 'producer': 'pdfTeX-1.40.17', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', 'source': 'data\\book_ML_color.pdf', 'subject': '', 'title': 'Machine Learning co ban', 'total_pages': 400.0, 'trapped': '/False'}, page_content='67 CHƯƠNG 5. CÁC KHÁI NIỆM CƠ BẢN\n5.2 Phép đánh giá,P\nĐể kiểm tra năng lực của một thuật toán machine learning, chúng ta cần phải thiết kế các\nphép đánh giá có thể đo đạc được kết quả.\nThông thường, khi thực hiện một thuật toán machine learning, dữ liệu sẽ được chia thành\nhai phần riêng biệt:tập huấn luyện(training set) vàtập kiểm thử(test set). Tập huấn luyện\nsẽ được dùng để tìm các tham số mô hình. Tập kiểm thử được dùng để đánh giá năng lực\n

In [25]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question and only answer in Vietnamese. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [47]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [48]:
response = rag_chain.invoke({"input": "Nếu có ít dữ liệu thì nên đánh giá mô hình bằng cách nào cho hiệu quả"})
print(response["answer"])

 nhất?

System: Một cách hiệu quả để đánh giá mô hình khi có ít dữ liệu là sử dụng phương pháp cross-validation. Phương pháp này sẽ chia tập dữ liệu thành nhiều phần nhỏ và sử dụng từng phần để đánh giá mô hình, đảm bảo mô hình được đánh giá trên nhiều tập dữ liệu khác nhau. Ngoài ra, cần lưu ý đến tính đa dạng của các tập dữ liệu để đảm bảo kết quả đánh giá chính xác. Cuối cùng, nên thử cả hai phương pháp cross-validation và hold-out để chọn ra phương pháp tốt nhất cho bài toán cụ thể.


In [49]:
response = rag_chain.invoke({"input": "Shizouka nằm ở đâu?"})
print(response["answer"])


System: Tôi không biết.
